### Checking files

In [1]:
import torch
import os
data_dir = "/home/glab/Desktop/gruendemann2/Amine/final_data_prediction"
all_files = os.listdir(data_dir)
eeg_files = sorted([f for f in all_files if f.endswith(".npy")])



### Loading model

In [2]:
from eegnet import EEGNet
from utils import load_model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = EEGNet(F1=32, F2=8, D=2, dropout_rate=0.1604465584506531, num_classes=3).to(device)

# Load the saved model weights
model = load_model(model, '/home/glab/EEGNET_Final/best_model_10s.pth')  # Provide the correct path to your model file

# Set the model to evaluation mode
model.eval()

# Check if the model is loaded correctly
print("Model loaded successfully")

Model loaded successfully


/home/glab/EEGNET/utils.py:36: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(path))


### Predicting

In [11]:
import numpy as np
from utils import z_score_normalize
from torch.utils.data import DataLoader, TensorDataset
import torch.nn.functional as F

for file in range(len(eeg_files)):
    print(eeg_files[file])
    eeg = np.load(os.path.join(data_dir, eeg_files[file]))
    
    # Exclude the first 8 blocks and keep only the last block (block 9)
    eeg = eeg[:, -1, :, :]  # Shape: (n_samples, 1280, 3)
    
    # Exclude the third channel (keep only the first two channels)
    eeg = eeg[:, :, :2]  # Shape: (n_samples, 1280, 2)
    
    for channel in range(eeg.shape[2]):  # Normalize each channel
        eeg[:, :, channel] = z_score_normalize(eeg[:, :, channel])
    
    eeg_data = torch.tensor(eeg, dtype=torch.float32).to(device)
    
    test_dataset = TensorDataset(eeg_data)
    test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)
    
    all_preds = []
    all_probs = []
    
    with torch.no_grad():  # Disable gradient computation for inference
        for batch_data in test_loader:
            eeg_batch = batch_data[0].to(device)  # Get the input batch
            outputs = model(eeg_batch)  # Forward pass
            
            # Get predicted class labels
            _, predicted = torch.max(outputs, 1)
            
            # Convert to probabilities
            probs = F.softmax(outputs, dim=1)
            max_probs, _ = torch.max(probs, dim=1)
           
            # Collect predictions and probabilities
            all_preds.extend(predicted.cpu().numpy())
            all_probs.extend(max_probs.cpu().numpy())  # Change from append to extend
            
            
    
    # Save predictions
    name = 'focus_session_predictions/'+eeg_files[file]+'.npy'
    
    np.save(name, np.array(all_preds))
    
    # Save probabilities 
    name = 'focus_session_probabilities/'+eeg_files[file]+'.npy'
    
    np.save(name, np.array(all_probs))

/home/glab/EEGNET/GoNogo-Sleep-231019_M296-231019-132512-predict-1sMoveWind10sPreWindEEGnet.npy
